In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
        .builder \
        .appName("Aula Interativa 2 - ML") \
        .getOrCreate()

spark.version

'3.5.1'

In [2]:
titanic_df = spark.read.csv('C:\\Users\\Matheus Poletto\\Desktop\\Cientista de Dados\\POS XP\\BOOTCAMP CDD\\MÓDULO 2\\2a aula interativa\\titanic.csv', header='True', inferSchema='True')

titanic_df.printSchema()



root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [3]:
titanic_df.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| NULL|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| NULL|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| NULL|       S|
|          6|       0|     3|    Moran, Mr. James|  male|NULL|    0|    0|      

In [4]:
# demostrando o sex_indexer
from pyspark.ml.feature import StringIndexer, OneHotEncoder

sex_indexer = StringIndexer(inputCol='Sex', outputCol='SexIndexer')
sex_indexer.fit(titanic_df).transform(titanic_df).select("PassengerId", "Sex","SexIndexer").show(10)

+-----------+------+----------+
|PassengerId|   Sex|SexIndexer|
+-----------+------+----------+
|          1|  male|       0.0|
|          2|female|       1.0|
|          3|female|       1.0|
|          4|female|       1.0|
|          5|  male|       0.0|
|          6|  male|       0.0|
|          7|  male|       0.0|
|          8|  male|       0.0|
|          9|female|       1.0|
|         10|female|       1.0|
+-----------+------+----------+
only showing top 10 rows



In [5]:
# demostrando o sex_encoder

sex_encoder = OneHotEncoder(inputCol='SexIndexer', outputCol='SexVector')

sex_indexer_model = sex_indexer.fit(titanic_df).transform(titanic_df)

# https://stackoverflow.com/questions/42295001/how-to-interpret-results-of-spark-onehotencoder
# primeiro valor: tamanho do vetor
# segundo valor: índices dos valores que não são zero
# terceiro valor: valores que não são zero
sex_encoder.fit(sex_indexer_model).transform(sex_indexer_model).select("PassengerId", "Sex","SexVector").show(10)

+-----------+------+-------------+
|PassengerId|   Sex|    SexVector|
+-----------+------+-------------+
|          1|  male|(1,[0],[1.0])|
|          2|female|    (1,[],[])|
|          3|female|    (1,[],[])|
|          4|female|    (1,[],[])|
|          5|  male|(1,[0],[1.0])|
|          6|  male|(1,[0],[1.0])|
|          7|  male|(1,[0],[1.0])|
|          8|  male|(1,[0],[1.0])|
|          9|female|    (1,[],[])|
|         10|female|    (1,[],[])|
+-----------+------+-------------+
only showing top 10 rows



In [6]:
from pyspark.ml.feature import VectorAssembler

#assembler = VectorAssembler(inputCols=['SexVector','Age','Pclass', 'Fare'], outputCol='features')
#assembler = VectorAssembler(inputCols=['Age', 'Pclass', 'Fare', 'SexVector'], outputCol='features')
assembler = VectorAssembler(inputCols=['Age','SexVector'], outputCol='features')






In [7]:
from pyspark.ml.classification import DecisionTreeClassifier

classifier = DecisionTreeClassifier(labelCol='Survived', featuresCol='features')

In [8]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=[sex_indexer, sex_encoder, assembler, classifier])
#pipeline = Pipeline(stages=[assembler, classifier])


In [9]:
train_data, test_data = titanic_df.randomSplit([0.7, 0.3])

In [10]:
predictSurvivedModel = pipeline.fit(train_data)

Py4JJavaError: An error occurred while calling o164.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 18.0 failed 1 times, most recent failure: Lost task 0.0 in stage 18.0 (TID 14) (DESKTOP-C2A46PP executor driver): org.apache.spark.SparkException: [FAILED_EXECUTE_UDF] Failed to execute user defined function (`VectorAssembler$$Lambda$3718/1802796138`: (struct<Age:double,SexVector:struct<type:tinyint,size:int,indices:array<int>,values:array<double>>>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>).
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:198)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNe

In [11]:
mean_age = titanic_df.agg({'Age': 'mean'}).collect()[0][0]
mean_age

29.69911764705882

In [12]:
titanic_df = titanic_df.fillna(mean_age, subset=['Age'])

In [13]:
train_data, test_data = titanic_df.randomSplit([0.7, 0.3])
predictSurvivedModel = pipeline.fit(train_data)

In [14]:
predictions = predictSurvivedModel.transform(test_data)
predictions.select('passengerId', 'fare', 'pclass', 'age', 'sex', 'rawPrediction', 'prediction').show(50)



+-----------+-------+------+-----------------+------+-------------+----------+
|passengerId|   fare|pclass|              age|   sex|rawPrediction|prediction|
+-----------+-------+------+-----------------+------+-------------+----------+
|          2|71.2833|     1|             38.0|female| [39.0,132.0]|       1.0|
|          4|   53.1|     1|             35.0|female| [39.0,132.0]|       1.0|
|          6| 8.4583|     3|29.69911764705882|  male| [329.0,63.0]|       0.0|
|          7|51.8625|     1|             54.0|  male| [329.0,63.0]|       0.0|
|         10|30.0708|     2|             14.0|female|   [9.0,20.0]|       1.0|
|         13|   8.05|     3|             20.0|  male| [329.0,63.0]|       0.0|
|         14| 31.275|     3|             39.0|  male| [329.0,63.0]|       0.0|
|         21|   26.0|     2|             35.0|  male| [329.0,63.0]|       0.0|
|         23| 8.0292|     3|             15.0|female|   [9.0,20.0]|       1.0|
|         24|   35.5|     1|             28.0|  male

In [15]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol='Survived', predictionCol='prediction', metricName='accuracy')
accuracy = evaluator.evaluate(predictions)

accuracy



0.7741935483870968

In [16]:
decisionTreeModel = predictSurvivedModel.stages[-1]

In [17]:
decisionTreeModel.depth

5

In [18]:
decisionTreeModel.toDebugString

'DecisionTreeClassificationModel: uid=DecisionTreeClassifier_de2424bfb67b, depth=5, numNodes=15, numClasses=2, numFeatures=2\n  If (feature 1 in {1.0})\n   If (feature 0 <= 9.5)\n    If (feature 0 <= 3.5)\n     Predict: 1.0\n    Else (feature 0 > 3.5)\n     Predict: 0.0\n   Else (feature 0 > 9.5)\n    Predict: 0.0\n  Else (feature 1 not in {1.0})\n   If (feature 0 <= 21.5)\n    If (feature 0 <= 3.5)\n     Predict: 0.0\n    Else (feature 0 > 3.5)\n     If (feature 0 <= 17.5)\n      Predict: 1.0\n     Else (feature 0 > 17.5)\n      If (feature 0 <= 18.5)\n       Predict: 0.0\n      Else (feature 0 > 18.5)\n       Predict: 1.0\n   Else (feature 0 > 21.5)\n    Predict: 1.0\n'

In [ ]:
assembler.getInputCols()


['Age', 'SexVector']

In [19]:
decisionTreeModel.featureImportances

SparseVector(2, {0: 0.0943, 1: 0.9057})

In [20]:

list(zip(assembler.getInputCols(), decisionTreeModel.featureImportances))

[('Age', 0.09430843553683897), ('SexVector', 0.905691564463161)]

In [21]:
assembler.getInputCols()

['Age', 'SexVector']

In [ ]:
basePath = "C:/Users/pedro.guerra/Downloads/XPE/cientista-dados/aula2"
decisionTreeModel.write().overwrite().save(basePath + "/model2")